# Simple Zinc Interaction Analysis Example

<img src="./figures/zinc_interaction.png" style="width: 300px;"/>

## Imports

In [1]:
from pyspark import SparkConf, SparkContext
from mmtfPyspark.datasets import groupInteractionExtractor
from mmtfPyspark.io import MmtfReader
from mmtfPyspark.webFilters import Pisces

## Configure Spark

In [2]:
conf = SparkConf().setMaster("local[*]") \
                  .setAppName("simpleZincInteractionDemo")

sc = SparkContext(conf = conf)

## Read PDB in MMTF format

In [3]:
path = "../../resources/mmtf_full_sample/"

pdb = MmtfReader.read_sequence_file(path, sc)

# Use only representative structures

In [4]:
seqId = 40
resolution = 2.0

pdb = pdb.filter(Pisces(seqId, resolution))

## Extract proteins with Zn interactions

In [5]:
finder = groupInteractionExtractor("ZN",3)

interactions = finder.getDataset(pdb).cache()

## List the top 10 residue types that interact with Zn

In [6]:
interactions.printSchema()

interactions.show(20)

print(f"Number of interactions: {interactions.count()}")

root
 |-- structureId: string (nullable = false)
 |-- residue1: string (nullable = false)
 |-- atom1: string (nullable = false)
 |-- element1: string (nullable = false)
 |-- index1: integer (nullable = false)
 |-- residue2: string (nullable = false)
 |-- atom2: string (nullable = false)
 |-- element2: string (nullable = false)
 |-- index2: integer (nullable = false)
 |-- distance: float (nullable = false)

+-----------+--------+-----+--------+------+--------+-----+--------+------+---------+
|structureId|residue1|atom1|element1|index1|residue2|atom2|element2|index2| distance|
+-----------+--------+-----+--------+------+--------+-----+--------+------+---------+
|       1FN9|      ZN|   ZN|      Zn|   730|     CYS|   SG|       S|    50|2.3709755|
|       1FN9|      ZN|   ZN|      Zn|   730|     CYS|   SG|       S|    53|2.3940797|
|       1FN9|      ZN|   ZN|      Zn|   730|     HIS|  NE2|       N|    70|2.2196307|
|       1FN9|      ZN|   ZN|      Zn|   730|     CYS|   SG|       S|    72

## Show the top 10 interacting groups

In [7]:
interactions.groupBy("residue2") \
            .count() \
            .sort("count", ascending = False) \
            .show(10)

+--------+-----+
|residue2|count|
+--------+-----+
|     HIS|   76|
|     CYS|   43|
|     HOH|   37|
|     GLU|   34|
|     ASP|   27|
|     ACT|   12|
|     TRP|    4|
|     VAL|    2|
|     LYS|    2|
|      CL|    1|
+--------+-----+



## Terminate Spark

In [8]:
sc.stop()